In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import h5py
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipfx.spike_detector as spkd
from ipfx.dataset.create import create_ephys_data_set
from ipfx.utilities import drop_failed_sweeps
from ipfx.epochs import get_stim_epoch
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.io import savemat
from pynwb import NWBHDF5IO
from helpers.features_100Hz_traces import get_time_voltage_current_currindex0, extract_spike_features, get_cell_features

In [3]:
root_path = 'C:/Users/yanez/Desktop/rehack/data/mouse m1'
data_path = root_path + '/raw/ephys/'
save_path = root_path + '/processed/ephys/'

In [40]:
meta_data_file_path = root_path + '/m1_patchseq_meta_data.csv'
cells = pd.read_csv(meta_data_file_path, sep='\t', index_col=0)
cells = cells[cells['Traced'] == 'y']

In [41]:
#create indices for excitatory and inhibitory cells
inhibitory_index = (cells['RNA family'] == 'Lamp5') | (cells['RNA family'] == 'Pvalb') | \
                    (cells['RNA family'] == 'Sncg') | (cells['RNA family'] == 'Sst') | \
                    (cells['RNA family'] == 'Vip') | (cells['Cell'] == '20190606_sample_7') | \
                    (cells['Cell'] == '20190905_sample_1')

no_dendrite_index = (cells['Cell'] != '20180921_sample_3')

inhibitory_index = inhibitory_index * no_dendrite_index

In [42]:
metadata = cells[inhibitory_index]
metadata

,Cell,Slice,Date,Sample,Mouse,Mouse date of birth,Mouse age,Mouse gender,Mouse genotype,Targeted layer,...,Length (bp),Yield (pg/µl),User,Hold Time (min),Soma depth (µm),Soma depth (4x),Cortical thickness (4x),Cortical thickness (µm),Traced,Exclusion reasons
Number,,,,,,,,,,,,,,,,,,,,,
80,20180213_sample_5,20180213_slice_5,2018-02-13,sample 5,mouse_JDVRZ,2017-11-30,75,M,Pvalb-Cre/wt; Ai9/wt,6,...,2073,240.87,Fede,30.0,1250.3,443.7,601.1,1659.036,y,NaN
81,20180215_sample_1,20180215_slice_1,2018-02-15,sample 1,mouse_LBJDN,2017-11-30,77,F,Pvalb-Cre/wt; Ai9/wt,5,...,2153,85.85,Fede,40.0,920.1,340.9,615.4,1698.504,y,NaN
91,20180306_sample_1,20180306_slice_1,2018-03-06,sample 1,mouse_YMIOM,2018-01-10,55,M,Pvalb-Cre/wt; Ai9/wt,2/3,...,1629,110.05,Fede,40.0,441.4,152.6,626.5,1729.140,y,NaN
121,20180327_sample_1,20180327_slice_1,2018-03-27,sample 1,mouse_XECLH,2018-01-22,64,F,Pvalb-Cre/wt; Ai9/wt,5,...,2347,199.59,Fede,35.0,596,220.9,509.5,1406.220,y,NaN
124,20180327_sample_4,20180327_slice_4,2018-03-27,sample 4,mouse_XECLH,2018-01-22,64,F,Pvalb-Cre/wt; Ai9/wt,5,...,2146,170.38,Matteo,48.0,753.7,272.7,529.0,1460.040,y,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,20191107_sample_3,20191107_slice_3,2019-11-07,Sample 3,mouse_WANEU,2019-08-21,78,F,Sst-IRES-Cre/wt; Ai9/wt,5,...,2300,131,Fede,40.0,354.4,130.3,488.8,1349.088,y,NaN
1309,20191107_sample_4,20191107_slice_4,2019-11-07,Sample 4,mouse_WANEU,2019-08-21,78,F,Sst-IRES-Cre/wt; Ai9/wt,5,...,1535,116.05,Fede,49.0,380.9,140.2,473.7,1307.412,y,NaN
1313,20191114_sample_1,20191114_slice_1,2019-11-14,Sample 1,mouse_YAVWS,2019-08-29,77,F,Viaat-Cre/wt; Ai9/wt,1,...,2303,179.79,Fede,31.0,115.05,44.3,581.0,1603.560,y,NaN


In [44]:
metadata["Cell"].values

array(['20180213_sample_5', '20180215_sample_1', '20180306_sample_1',
       '20180327_sample_1', '20180327_sample_4', '20180327_sample_5',
       '20180404_sample_2', '20180404_sample_5', '20180410_sample_9',
       '20180411_sample_4', '20180411_sample_5', '20180417_sample_1',
       '20180510_sample_2', '20180515_sample_1', '20180524_sample_9',
       '20180621_sample_7', '20180628_sample_2', '20180704_sample_5',
       '20180705_sample_1', '20180711_sample_1', '20180711_sample_3',
       '20180712_sample_14', '20180717_sample_3', '20180717_sample_4',
       '20180719_sample_1', '20180720_sample_5', '20180725_sample_9',
       '20180817_sample_2', '20180817_sample_3', '20180817_sample_4',
       '20180820_sample_1', '20180820_sample_4', '20180820_sample_5',
       '20180820_sample_7', '20180822_sample_1', '20180822_sample_2',
       '20180822_sample_3', '20180822_sample_4', '20180822_sample_6',
       '20180822_sample_7', '20180828_sample_2', '20180828_sample_3',
       '20180828_sa

In [28]:
warnings.filterwarnings("ignore") # It complains about some namespaces, but it should work.
m1_nwb_metadata, m1_nwb_paths = [], []
for root, dirs, files in os.walk(data_path):
    if files:
        for file in files:
            if file.endswith('.nwb'):
                m1_nwb_tmp = NWBHDF5IO(root + '/' + file, 'r', load_namespaces=True).read()
                if (metadata["Cell"].values==m1_nwb_tmp.session_id).any():
                    print(root + '/' + file)   
                    m1_nwb_metadata.append(m1_nwb_tmp)
                    m1_nwb_paths.append(root + '/' + file)

C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-ANUPT/sub-mouse-ANUPT_ses-20180725-sample-9_slice-20180725-slice-9_cell-20180725-sample-9_icephys.nwb
C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-APLSV/sub-mouse-APLSV_ses-20180817-sample-2_slice-20180817-slice-2_cell-20180817-sample-2_icephys.nwb
C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-APLSV/sub-mouse-APLSV_ses-20180817-sample-3_slice-20180817-slice-3_cell-20180817-sample-3_icephys.nwb
C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-APLSV/sub-mouse-APLSV_ses-20180817-sample-4_slice-20180817-slice-4_cell-20180817-sample-4_icephys.nwb


KeyboardInterrupt: 

In [8]:
# Save utils because previous step takes too long
# # hf = h5py.File(save_path + 'm1_nwb_metadata.h5', 'w')
# # hf.create_dataset('m1_nwb_metadata', data=m1_nwb_metadata)
# # hf.close()
# with open(save_path + 'm1_nwb_paths.pickle', 'wb') as f:
#     pickle.dump(m1_nwb_paths, f)# 

In [45]:
with open(save_path + 'm1_nwb_paths.pickle', 'rb') as f:
    nwb_paths = pickle.load(f)
# m1_nwb_metadata = m1_nwb_utils['m1_nwb_metadata']
# m1_nwb_paths = m1_nwb_utils['m1_nwb_paths']

In [52]:
sorted(nwb_paths)

['C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-ANUPT/sub-mouse-ANUPT_ses-20180725-sample-9_slice-20180725-slice-9_cell-20180725-sample-9_icephys.nwb',
 'C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-APLSV/sub-mouse-APLSV_ses-20180817-sample-2_slice-20180817-slice-2_cell-20180817-sample-2_icephys.nwb',
 'C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-APLSV/sub-mouse-APLSV_ses-20180817-sample-3_slice-20180817-slice-3_cell-20180817-sample-3_icephys.nwb',
 'C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-APLSV/sub-mouse-APLSV_ses-20180817-sample-4_slice-20180817-slice-4_cell-20180817-sample-4_icephys.nwb',
 'C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-AVLEY/sub-mouse-AVLEY_ses-20190515-sample-6_slice-20190515-slice-6_cell-20190515-sample-6_icephys.nwb',
 'C:/Users/yanez/Desktop/rehack/data/mouse m1/raw/ephys/sub-mouse-AWOAY/sub-mouse-AWOAY_ses-20190829-sample-6_slice-20190829-slice-4_cell-20190829-sam

In [10]:
nwb_metadata = NWBHDF5IO(nwb_paths[cell_id], 'r', load_namespaces=True).read()
specimen_id_now = nwb_metadata.session_id
specimen_id_now

NameError: name 'cell_id' is not defined

In [ ]:
# Set variables to save
all_ephys_features = pd.DataFrame()
V, specimen_id, t_type = [], [], []

# Set stimulus duration (500 ms as in our data)
stim_duration = 0.5 

# Set time window before and after stimulus (100 ms as in our data)
t_no_stim = 0.1

# Define standardized number samples, make sure that sampling freq is above Nyquist (> 20kHz works)
t_standardized = np.arange(0, 0.7, 1/30000)

for cell_id in np.arange(len(metadata)):
    
    # Initialize variable to know if cell should be saved
    valid_cell = False
        
    # Get current specimen id
    nwb_metadata = NWBHDF5IO(nwb_paths[cell_id], 'r', load_namespaces=True).read()
    specimen_id_now = nwb_metadata.session_id

    # Load experiment for current specimen
    t, all_voltages, all_currents, _ = get_time_voltage_current_currindex0(nwb_metadata)
    
    # Initialize 100 Hz variable
    best_init_freq = np.inf

    for sweep_id in np.arange(len(all_currents)):

        # Compute variables in tVI format (s,mV,mA)
        v = all_voltages[:,sweep_id]
        i = all_currents[sweep_id]

        # Get current stim onset
        t_stim_onset  = t_no_stim
        t_stim_offset = t_stim_onset + stim_duration

        # Detect spikes
        putative_spikes = spkd.detect_putative_spikes(v, t, t_stim_onset, t_stim_offset)
        peaks = spkd.find_peak_indexes(v, t, putative_spikes, t_stim_offset)
        peak_indexes, spike_indexes = spkd.filter_putative_spikes(v, t, putative_spikes, peaks)

        # Check if it's a valid sweep (Freq>5Hz) and that V and I aren't missing
        if (len(spike_indexes) >= 2):

            # Calculate initial firing frequency in Hz
            init_freq = 1 / (t[spike_indexes[1]] - t[spike_indexes[0]])
            
            # Calculate the coefficient of variation of spike amplitudes
            trough_indexes = spkd.find_trough_indexes(v, t, spike_indexes, peak_indexes)
            spike_amp = v[spike_indexes] - v[trough_indexes.astype(int)]
            CV_amp = np.std(spike_amp) / np.mean(spike_amp)

            # Store the selected 100 Hz trace
            if np.abs(100-init_freq)<=np.abs(100-best_init_freq):
                
                if init_freq==best_init_freq:
                    if best_CV_amp >= CV_amp:
                        # update saved CV of spike amplitudes
                        best_CV_amp = CV_amp
                        # update saved sweep
                        best_sweep_id = sweep_id
                else:
                    # save initial firing rate
                    best_init_freq = init_freq
                    # save CV of spike amplitudes
                    best_CV_amp = CV_amp
                    # save corresponding sweep
                    best_sweep_id = sweep_id
                    
                # get sampling rate
                sampling_rate = nwb_metadata.stimulus['CurrentClampStimulusSeries000'].rate
                # get trace from 0ms to 500ms after stimulus onset
                idx_start = 0
                idx_mid1  = int((t_stim_onset+stim_duration)*sampling_rate)
                V_tmp1 = v[range(idx_start, idx_mid1)]
                # set number of samples after stimulus, add delta to   
                # avoid unwanted fluctuations directly after end of stimulus
                V_tmp2 = v[int(0.71*sampling_rate):] 
                # smooth decay 
                V_tmp2 = V_tmp2[v[idx_mid1] > V_tmp2]
                # concatenate voltages (end of experiment determined during standardization)
                V_crop = np.concatenate((V_tmp1, V_tmp2[:int(2*t_no_stim*sampling_rate)]))
                
                if int(0.7*sampling_rate)<=len(V_crop):
                    # standardize voltage trace
                    interp = interp1d(np.arange(0, len(V_crop))/sampling_rate, V_crop)
                    best_sweep = interp(t_standardized)
                    curr_best_sweep = np.array([i])
                    # set cell as valid (note by definition all met cells are valid)
                    valid_cell = True

    if valid_cell:
        
        features_now = get_cell_features(specimen_id_now, t_standardized, curr_best_sweep, 
                                         np.expand_dims(best_sweep, axis=1), 
                                         start=t_no_stim, end=t_no_stim+stim_duration)
        features_now.to_csv(save_path + 'features/' + str(specimen_id_now) + '.csv') 
        all_ephys_features = pd.concat([all_ephys_features, features_now], sort = True)
            
        t_type_now = metadata.iloc[cell_id, :]['RNA type']
        
        specimen_id.append(specimen_id_now)
        t_type.append(t_type_now)
        V.append(best_sweep)

        print("%4.f Specimen ID: %s | Sweep ID: %d | Init. freq.: %.2f Hz" % 
              (cell_id, specimen_id_now, best_sweep_id, best_init_freq))

In [ ]:
all_ephys_features.to_csv(save_path + 'm1_patchseq_ephys_features.csv') 

In [ ]:
V = np.vstack(V)
V.shape

In [ ]:
# with open(save_path + 'voltage_traces.pickle', 'wb') as f:
#     pickle.dump({"V": V, "specimen_id": specimen_id, "t_type": t_type}, f)

In [ ]:
savemat(save_path + 'voltage_traces.mat', \
    {"V": V, "specimen_id": specimen_id, "t_type": t_type})